In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Concatenate, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.metrics import Precision, Recall
from skimage.feature import graycomatrix, graycoprops
import cv2

C:\Users\Sushant\AppData\Local\Temp\ipykernel_4688\3943852633.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Function to load and preprocess image using Keras
def load_and_preprocess_image(image_path, target_size):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = tf.keras.applications.resnet50.preprocess_input(img_array)
    return img_array

# Function to extract image paths from a folder
def extract_image_paths(root_folder, folder):
    image_paths = []
    for subdir, _, files in os.walk(os.path.join(root_folder, folder)):
        for file in files:
            if file.lower().endswith('.jpg'):
                image_path = os.path.join(subdir, file)
                image_paths.append(image_path)
    return image_paths

# Replace these paths with your actual dataset folder and subfolders
root_folder_path = "Reduced_Dataset"
real_folder = "Real"
fake_folder = "Fake"

In [3]:
# Extract image paths for real and fake images
real_image_paths = extract_image_paths(root_folder_path, real_folder)
fake_image_paths = extract_image_paths(root_folder_path, fake_folder)

In [4]:
# Create labels (0 for real, 1 for fake)
real_labels = np.full(len(real_image_paths), "Real")
fake_labels = np.full(len(fake_image_paths), "Fake")

In [5]:
# Combine real and fake data
X = np.concatenate([real_image_paths, fake_image_paths], axis=0)
y = np.concatenate([real_labels, fake_labels], axis=0)
df = pd.DataFrame({'Image_Path': X, 'Nature': y})
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
print(df.tail())

                                              Image_Path Nature
21384  Reduced_Dataset\Fake\01_11__meeting_serious__F...   Fake
21385  Reduced_Dataset\Fake\01_26__outside_talking_pa...   Fake
21386  Reduced_Dataset\Real\14__walk_down_hall_angry....   Real
21387  Reduced_Dataset\Real\03__hugging_happy.mp4\fra...   Real
21388  Reduced_Dataset\Fake\03_09__outside_talking_st...   Fake


In [6]:
target_size = (224, 224)

# Create an ImageDataGenerator for real and fake images
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2)

# Create a flow_from_dataframe generator for training
train_generator = train_datagen.flow_from_dataframe(
    df,
    x_col='Image_Path',
    y_col='Nature',
    target_size=target_size,
    batch_size=8,
    class_mode='binary',
    subset='training'
)

# Create a flow_from_dataframe generator for validation
validation_generator = train_datagen.flow_from_dataframe(
    df,
    x_col='Image_Path',
    y_col='Nature',
    target_size=target_size,
    batch_size=8,
    class_mode='binary',
    subset='validation'
)



Found 17112 validated image filenames belonging to 2 classes.
Found 4277 validated image filenames belonging to 2 classes.


In [7]:
def extract_glcm_features(images):
    glcm_features = []
    for img in images:
        # Calculate GLCM
        glcm = graycomatrix(img, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
        # Extract properties from GLCM
        properties = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation']
        glcm_props = [graycoprops(glcm, prop).ravel()[0] for prop in properties]
        glcm_features.append(glcm_props)
    return np.array(glcm_features)

In [8]:
def extract_resnet_features(images):
    # Load pre-trained ResNet50 model
    base_model = ResNet50(weights='imagenet', include_top=False,input_shape=(224, 224, 3), pooling='avg')
    # Preprocess images for ResNet50
    preprocessed_images = tf.keras.applications.resnet50.preprocess_input(images)
    # Extract features using ResNet50
    features = base_model.predict(preprocessed_images)
    return features

In [9]:
def create_model(input_shape_resnet, input_shape_glcm, num_classes):
    # Define input layers for ResNet50 and GLCM features
    input_resnet = Input(shape=input_shape_resnet)
    input_glcm = Input(shape=input_shape_glcm)
    
    # Concatenate ResNet50 and GLCM features
    combined_features = Concatenate()([input_resnet, input_glcm])
    
    # Add a dense layer for classification
    dense_layer = Dense(64, activation='relu')(combined_features)
    output_layer = Dense(num_classes, activation='softmax')(dense_layer)
    
    # Define the model
    model = Model(inputs=[input_resnet, input_glcm], outputs=output_layer)
    
    return model

CheckPoint added

In [10]:
# Load and preprocess images
images = []
for path in X:
    img = cv2.imread(path)
    if img is not None:
        img = cv2.resize(img, target_size)
        images.append(img)
images = np.array(images)

# Extract ResNet50 features
resnet_features = extract_resnet_features(images)

# Extract GLCM features
glcm_features = extract_glcm_features(images)

# Define input shapes
input_shape_resnet = resnet_features.shape[1:]
input_shape_glcm = glcm_features.shape[1:]

# Define number of classes
num_classes = 2

# Create and compile model
model = create_model(input_shape_resnet, input_shape_glcm, num_classes)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

MemoryError: Unable to allocate 12.0 GiB for an array with shape (21389, 224, 224, 3) and data type float32

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', Precision(), Recall()])

In [ ]:
# callbacks
early_stopping = EarlyStopping(monitor='val_accuracy', patience=patience, min_delta=min_delta, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy', save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.001, verbose=1)

# Train the model
with tf.device('/GPU:0'):  # Ensure training is done on GPU
    history = model.fit(
        train_generator,
        epochs=50,  # Increase the number of epochs
        validation_data=validation_generator,
        callbacks=[early_stopping, model_checkpoint, reduce_lr],
        class_weight=class_weights_dict,
        verbose=1
    )

# Save the model
model.save('Resnet_GLCM.h5')

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

# Define paths to your data
test_data_dir = "Testing"  # Replace with your testing data path

# Define image dimensions (assuming you used these during training)
img_height, img_width = 224, 224

# Load your trained ResNet50 model
model = load_model("trained_resnet_model_real_fake_Optimized.h5")

# Define data generators for testing
test_datagen = ImageDataGenerator(rescale=1./255)  # Normalize pixel values

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=8,  # Adjust batch size as needed
    class_mode='binary'  # Adjust class mode based on your task (e.g., 'binary' for 2 classes)
)

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator)

# Print the test accuracy
print(f"Test Accuracy: {test_acc:.4f}")

# Additional analysis (optional)
# predictions = model.predict(test_generator)
# Analyze predictions based on your specific task (e.g., confusion matrix, visualization)
